# Loading a Linear Learner Model with Apache MXNet in Python

In the previous recipe, we ran a training job using the **SageMaker Python SDK**. In this recipe, we will use **Apache MXNet** and **Gluon** to load the model, extract its parameters, and perform predictions locally. If you are wondering what **Gluon** is and how it differs from **Apache MXNet**, **Gluon** is a high-level API for deep learning while **Apache MXNet** is the deep learning framework usually categorized with **TensorFlow** and **PyTorch**.

![Book Cover](../Extra/chap01/06.png)

That said, the objective of this recipe is to show that the model file uploaded to the **Amazon S3** bucket after the training step can be loaded and analyzed using **Apache MXNet** as shown in the image above.

### How to do it...

In [ ]:
%store -r model_data

In [ ]:
model_data

In [ ]:
import sagemaker 
session = sagemaker.Session()

In [ ]:
from sagemaker.s3 import S3Downloader

In [ ]:
S3Downloader.download(s3_uri=model_data,
                      local_path="tmp/",
                      sagemaker_session=session)

In [ ]:
!ls tmp

In [ ]:
!tar -xzvf tmp/model.tar.gz

In [ ]:
!unzip model_algo-1 

In [ ]:
import mxnet

from mxnet import gluon
from json import load as json_load 
from json import dumps as json_dumps

In [ ]:
sym_json = json_load(open('mx-mod-symbol.json')) 
sym_json_string = json_dumps(sym_json)

model = gluon.nn.SymbolBlock( 
    outputs=mxnet.sym.load_json(sym_json_string), 
    inputs=mxnet.sym.var('data'))

model.load_parameters('mx-mod-0000.params', allow_missing=True)

In [ ]:
model.initialize()

def mxnet_predict(x, model=model):
    return model(mxnet.nd.array([x]))[0].asscalar()

In [ ]:
mxnet_predict(42)

In [ ]:
def extract_weight_and_bias(model):
    params = model.collect_params()
    weight = params['fc0_weight'].data()[0].asscalar() 
    bias = params['fc0_bias'].data()[0].asscalar()
    
    return {
        "weight": weight, 
        "bias": bias
    } 

In [ ]:
weight_and_bias = extract_weight_and_bias(model)
weight_and_bias

In [ ]:
%store weight_and_bias

In [ ]:
%%bash

rm -f additional-params.json
rm -f manifest.json
rm -f model_algo-1
rm -f mx-mod-symbol.json
rm -f mx-mod-0000.params

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)